In [5]:
import os
import re
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pytesseract
from PIL import Image
import string
from nltk.corpus import stopwords
import pandas as pd
from tabulate import tabulate
import easyocr
from concurrent.futures import ThreadPoolExecutor

In [6]:
#Transformer library installed and did not go through, prob deapricated, easyocr the upgraded version.

In [7]:
# Set up Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Path to the folder containing PNG images
folder_path = "C:/Users/hseit/Desktop/SummerMCREU/images/"
amount = 0
extracted_data = []

In [10]:
# Keywords for feature detection
keywords = {
    "product_title": ["product name", "name", "product"],
    "directions": ["directions", "usage", "how to use"],
    "supplements_or_elements": ["supplements", "elements", "ingredients"],
    "warnings": ["warnings", "cautions", "precautions"]
}

# Load the OCR reader with the Transformer-based model
reader = easyocr.Reader(['en'])
print()

# Iterate over the PNG images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        amount += 1
        print("File found: " + str(amount))
        # Load the label image
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        print()

         # Preprocess the image
        resized_image = cv2.resize(image, (800, 600))  # Resize the image
        normalized_image = cv2.normalize(resized_image, None, 0, 255, cv2.NORM_MINMAX)  # Normalize pixel values
        denoised_image = cv2.fastNlMeansDenoisingColored(normalized_image, None, 10, 10, 7, 21)  # Denoise the image
        gray = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

        # Perform text detection
        contours, _ = cv2.findContours(gray.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        text_regions = []
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w > 10 and h > 10:  # Filter out small contours
                text_regions.append((x, y, w, h))

        # Find the contour with maximum area
        max_contour = max(text_regions, key=lambda r: r[2] * r[3])
        x, y, w, h = max_contour

        # Crop the image using the contour coordinates
        cropped_image = gray[y:y+h, x:x+w]

        # Perform text recognition on the cropped image using the OCR reader
        extracted_text = reader.readtext(cropped_image)

        # Extract the recognized text from the OCR results
        generated_text = " ".join([text[1] for text in extracted_text])

        # Match keywords to determine the feature type
        extracted_dict = {}
        for feature, feature_keywords in keywords.items():
            for keyword in feature_keywords:
                if keyword.lower() in generated_text.lower():
                    extracted_dict[feature] = generated_text
                    break

        # Store the extracted data
        data = {
            "image_path": image_path,
            "product_title": extracted_dict.get("product_title", ""),
            "directions": extracted_dict.get("directions", ""),
            "supplements_or_elements": extracted_dict.get("supplements_or_elements", ""),
            "warnings": extracted_dict.get("warnings", "")
        }
        extracted_data.append(data)

        # Display the results
        for region in text_regions:
            x, y, w, h = region
            cv2.rectangle(resized_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Print extracted text
        print("Extracted Text:")
        print(generated_text)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.



File found: 34

Extracted Text:
essential Waitrose british beef diced steak SA sooge
File found: 35

Extracted Text:
FBA Box 1 of 1 Ib SHIP FROM SHIP To James Bond FBA dnest-sta012 333 Boren Ave N Amazon com Services Inc Seattle WA 98109 4255 Anson Blvd United States Whitestown IN 46075-4412 United States FBA (10p0n19 1OS5 AMA FBA1SJDICSRIUOOOOO1 Mixed SKUs KM-SSHL-KJIN Qty PLEASE LEAVE THIS LABEL UNCOVERED
File found: 36

Extracted Text:
ALUMINUM FREE Oed <Spicer PURE SPORT HICH ENDURANCE€ DEODORANT NET WT 24 0Z (68g)
File found: 37

Extracted Text:
CONTAINS OpOR_FIGHTING "ATOMIC ROBOTS" THAT "SHOOT LASERS" 'Svith YOUR STENCH MONSTERS AND REPLACES THEM FRESH, CLEAN, MASCULINE SCENT ELVES: 48 HOUR ODOR PROTECTION DIRECTIONS: Twist Up product: Apply tO underarms only: Use daily for best results MARNINGS: DO NOT APPLY TO BROKEN SKIN: IF RASH OR IRRITATION DEVELOPS DISCONTINUE USE: USE ONLY AS DIRECTED: KEEP Out oF REACH OF CHILDREN_ DUGREDIENTS: DIPAOPYLENE GLYCOL, WATER, PROPYLENE GLYC


Extracted Text:
Wonderful_ Nutritor per 533 PISTACHIOS Aebvins Sige 7204882 cup with pinch of salt  each Segingihout shells) WVith just (289 servingof Lightly Salted Wonderful 7idi nnn  160 Pistachios 1s Ea e Bodium 83ieiss perfectly seasoned; tasty snack_ Daly Valuc The delicious roasted 34 17% comes through in every pistachio Total Faed Far159 32 crack open Lightly Salted 09 Nongerfui Pistachios are your Ted Fat 3 59 perfect snacking salt-ernative aed en 02 WonderfulPistachios.com cholesterel Omg Sodiun EomO 3% Scientific evidence suggests Total crbehydrante 89 3% but does not prove that ` FiDE 39 Te eating ounces per day 34g25 29 Keactfy of most nuts, such rouoc5 0g Addeq Sugars 62 pistachios, as part of diet low Protein 69 129 saturated fat and cholesterol may reduce the risk of heart disease See Omcg nutrition information for fat content: 2021 Wonderful Pistachios & Almonds LLC AlI Rights Reserved_ WONDERFUL, PLANT PROTEIN, the accompanying logos and trade dress areatrademarks of

Extracted Text:
laws Classic Rutalo Cnips Net 10 0Z 297.6 @i Size Famiv
File found: 61

Extracted Text:
BURGER PLANT-BASFD 20G. No pe8] BEYOND MEAT BEYOND Pat TIEc GLUIEN
File found: 62

Extracted Text:
Wutronacs 1 serving per container 9 1 Serving size Pkg. Dry(569) 3 2 8 Amount Galories Per 210 6 83 % Daily Value 8 Total Fat 1g 083 Saanserad 8909 0% Cholesterol Omg Sodium 590mg 25% Total Carbohydrate 459 15% Dietary Fiber 4g 14% 3 Total Sugars 4g 3 4 Includes Og Added Sugars 0% 3 Proteln 59 vil; 2mcg 6% Potas. 320mg 10% 3 Iron 65% Caicin 250rng 25% DilyViluo  Icllg yau m4#much nulient 07 #GctIr) Gocdmnnaner daily diet: 2,0C0 calonesa&7 Fnr nerarl nutritlon adjce. 1 1 0 ! 2 2 I 3 8 1 8 2 1 8 0 8 1 1 72mg
File found: 63

Extracted Text:
To: Company Name P Address, City ZIP. State PRIORITY MAIL FROM: Company Name Address , City ZIP State LOT NUMBER: REF NUMBER: SHIP DATE WEIGHT: HANDLE Item nr 1 WTH FRAGLE CARE Delivervy instruction:
File found: 64

Extracted Text:
prime
File found: 65


In [11]:
# Keywords for feature detection
keywords = {
    "product_title": ["product name", "name", "product"],
    "directions": ["directions", "usage", "how to use"],
    "supplements_or_elements": ["supplements", "drug facts", "ingredients"],
    "warnings": ["warnings", "cautions", "precautions"]
}

amount = 0
extracted_data = []
# Faster method for process_image algorithm using threading
def process_image(image_path):
    global amount
    amount += 1
    print("File found: " + str(amount))

    # Load the label image
    image = cv2.imread(image_path)

    # Preprocess the image
    resized_image = cv2.resize(image, (600, 600))  # Resize the image
    normalized_image = cv2.normalize(resized_image, None, 0, 255, cv2.NORM_MINMAX)  # Normalize pixel values
    denoised_image = cv2.fastNlMeansDenoisingColored(normalized_image, None, 10, 10, 7, 21)  # Denoise the image
    gray = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

    # Perform text detection
    contours, _ = cv2.findContours(gray.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    text_regions = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 10 and h > 10:  # Filter out small contours
            text_regions.append((x, y, w, h))

#     if text_regions:
    # Find the contour with maximum area
    max_contour = max(text_regions, key=lambda r: r[2] * r[3])
    x, y, w, h = max_contour

    # Crop the image using the contour coordinates
    cropped_image = gray[y:y+h, x:x+w]

    # Perform text recognition on the cropped image
    extracted_text = pytesseract.image_to_string(cropped_image).strip()

    # Match keywords to determine the feature type
    extracted_dict = {}
    for feature, feature_keywords in keywords.items():
        for keyword in feature_keywords:
            if keyword.lower() in extracted_text.lower():
                extracted_dict[feature] = extracted_text
                break

    # Store the extracted data
    data = {
        "image_path": image_path,
        "extracted_text": extracted_text,
        "product_title": extracted_dict.get("product_title", ""),
        "directions": extracted_dict.get("directions", ""),
        "supplements_or_elements": extracted_dict.get("supplements_or_elements", ""),
        "warnings": extracted_dict.get("warnings", "")
    }
    extracted_data.append(data)

    # Display the results
    for region in text_regions:
        x, y, w, h = region
        cv2.rectangle(resized_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        print("Extracted Text:")
        print(extracted_text)

def folderOCR(directory_path):
    image_paths = [os.path.join(directory_path, filename) for filename in os.listdir(directory_path)
                   if filename.endswith((".jpg", ".png"))]

    with ThreadPoolExecutor() as executor:
        executor.map(process_image, image_paths)

# Run the folderOCR function
folderOCR(folder_path)

File found: 1
File found: 2
File found: 3
File found: 4
File found: 5
File found: 6
File found: 7
File found: 8
File found: 9
File found: 10
File found: 11
File found: 12
File found: 13
File found: 14
File found: 15
File found: 16
Extracted Text:
FBA Box 1 of 1 - 1Ib

SHIP FROM SHIP TO

James Bond FBA. dnest+sta012

333 Boren Ave N Amazon.com Services, Inc.

Seattle, WA 98109 4255 Anson Blvd

United States Whitestown, IN 46075-4412
United States

FBA (10/30/19 10:55 AM) - 1

ae
pea

FBA15JD9C5R9U000001
Mixed SKUs pnd
| |

KM-SSHL-KJON
Qty 1

PLEASE LEAVE THIS LABEL UNCOVERED
File found: 17
Extracted Text:
BAUSCH # LOMB

Eye Relief.

LUBRICANT EYE DROPS

e Fast relief from
dry eyes

e Replenishes tears

° Rejuvenates eyes
File found: 18
Extracted Text:
per container
4 pack (509) (4 biscuits)

—_7—i
Calories _ 230
File found: 19
Extracted Text:

File found: 20
Extracted Text:
ong racks a
Active ingredient Purpose

(per tablet)
Calcium Carbonate USP 100mg.
Usesiaieves = nearbum
atid malge

Extracted Text:

Extracted Text:
jon Facts

1 serving per container

Serving size 1 Pkg. Dry (56a)

Gaiories 210

ed By
pra Foods, LLC
1d, CO 80537 USA
s, Organi
s Pumpki

Bear

Cholesterol omg
Sodium 590mg |
Total Carbohydrate 459 15%
Dietary Fiber 4g
Total Sugars 4g Z
includes 0g Added Sugars 0%

t.0 2meg 6% « Potas. 9209 10%
ron amg 659% «Calcium 250mg 259%
ae ate ow uch aru
Extracted Text:
ACQUA DI

GIORGIO ARMANI

~~ ABSOLU

TATOMIAICYQW
Extracted Text:

Extracted Text:

Extracted Text:
KILLS 99.9% OF GERMS
THAT CAUSE BAD BREATH,
PLAQUE & GINGIVITIS |

ANTISEPTIC

LISTERINE
COOL MINT

FOR A FRESHER &
CLEANER MOUTH THAN
BRUSHING ALONE

yn
= -- -
2
a i SS-
Extracted Text:
Es
eg 3
a

|

2 at
ah
er Sane
revere
RedState
"ecarent)

met
ee

31 GRAM,

eg Try
Wer rt Btn
aac me

GOLD STHNARG Loe
inet
Extracted Text:
S4

31 GRAI 30 SEcoNps
‘ar Thou
tire asses
ret

For bast results, mx up your shake 30-60 nines at
you workout, ch have it as en anytime sneck in aur
bela-cee, Foh-pretein cit.



In [12]:
# Create a DataFrame from the extracted data
df = pd.DataFrame(extracted_data)

# Define a function to truncate long strings
def truncate_string(text, max_length=30):
    if len(text) > max_length:
        return text[:max_length] + '...'
    return text

# Apply string truncation to the DataFrame
df = df.applymap(truncate_string)

# Display the DataFrame in table format
print("\nExtracted Data:")
print(df.to_string(index=False))


Extracted Data:
                       image_path                             extracted_text                       product_title                          directions             supplements_or_elements                            warnings
C:/Users/hseit/Desktop/SummerM...        FBA Box 1 of 1 - 1Ib\n\nSHIP FRO...                                                                                                                                                
C:/Users/hseit/Desktop/SummerM...      BAUSCH # LOMB\n\nEye Relief.\n\nLU...                                                                                                                                                
C:/Users/hseit/Desktop/SummerM...         per container\n4 pack (509) (4 ...                                                                                                                                                
C:/Users/hseit/Desktop/SummerM...                                                                  